In [2]:
# 잡코리아 채용공고 크롤링
import requests
parsed= []
job = requests.get("https://www.wanted.co.kr/api/v4/jobs?1699925253447&country=kr&tag_type_ids=518&job_sort=job.latest_order&locations=all&years=-1").json()
job





{'model_status': None,
 'links': {'prev': None,
  'next': '/api/v4/jobs?country=kr&tag_type_ids=518&locations=all&years=-1&limit=20&offset=20&job_sort=job.latest_order'},
 'is_callable_external_job': False,
 'data_model': None,
 'data': [{'status': 'active',
   'reward': {'formatted_total': '100만원',
    'formatted_recommender': '50만원',
    'formatted_recommendee': '50만원'},
   'is_like': False,
   'is_bookmark': False,
   'company': {'id': 1531,
    'industry_name': '교육',
    'application_response_stats': {'avg_rate': 71.25,
     'level': 'low',
     'delayed_count': 0,
     'avg_day': None,
     'remained_count': 0,
     'type': 'applied_reply_done'},
    'name': '민병철교육그룹'},
   'title_img': {'origin': 'https://static.wanted.co.kr/images/company/1531/hnp8mpjy0g1arff5__1080_790.png',
    'thumb': 'https://static.wanted.co.kr/images/company/1531/hnp8mpjy0g1arff5__400_400.png'},
   'compare_country': True,
   'due_time': '2023-12-10',
   'like_count': 0,
   'id': 192877,
   'logo_img': {'o